<a href="https://colab.research.google.com/github/Baijiong-Lin/LoRA-Torch/blob/main/examples/linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/microsoft/LoRA
!pip install git+https://github.com/Baijiong-Lin/LoRA-Torch

  Cloning https://github.com/microsoft/LoRA to /tmp/pip-req-build-e5ulip2v
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/LoRA /tmp/pip-req-build-e5ulip2v
  Resolved https://github.com/microsoft/LoRA to commit 998cfe4d351f4d6b4a47f0921dec2397aa0b9dfe
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/Baijiong-Lin/LoRA-Torch to /tmp/pip-req-build-l221wgno
  Running command git clone --filter=blob:none --quiet https://github.com/Baijiong-Lin/LoRA-Torch /tmp/pip-req-build-l221wgno
  Resolved https://github.com/Baijiong-Lin/LoRA-Torch to commit 4b550558ff1c7ef6bd1009b80d30e52069396f69
  Preparing metadata (setup.py) ... done


In [2]:
import torch, loralib, loratorch, copy
import torch.nn as nn

In [3]:
model_lib = loralib.Linear(5, 6, r=4, lora_alpha=1)
model_torch = loratorch.Linear(5, 6, r=4, lora_alpha=1)

model_lib.weight.data = copy.deepcopy(model_torch.weight.data)
model_lib.lora_A.data = copy.deepcopy(model_torch.w_lora_A.data)
model_lib.lora_B.data = copy.deepcopy(model_torch.w_lora_B.data)
model_lib.bias.data = copy.deepcopy(model_torch.bias.data)

In [4]:
loralib.mark_only_lora_as_trainable(model_lib)
loratorch.mark_only_lora_as_trainable(model_torch)

In [5]:
optimizer_lib = torch.optim.SGD(model_lib.parameters(), lr=0.1)
optimizer_torch = torch.optim.SGD(model_torch.parameters(), lr=0.1)

for _ in range(3):
    model_lib.train()
    model_torch.train()
    x = torch.rand(2, 5)

    loss1 = model_lib(x).sum()
    optimizer_lib.zero_grad()
    loss1.backward()
    optimizer_lib.step()

    loss2 = model_torch(x).sum()
    optimizer_torch.zero_grad()
    loss2.backward()
    optimizer_torch.step()

    # for k, v in model_lib.named_parameters():
    #     print(k, v.grad)

    # for k, v in model_torch.named_parameters():
    #     print(k, v.grad)
    print(torch.isclose(model_lib.lora_A.grad, model_torch.w_lora_A.grad))
    print(torch.isclose(model_lib.lora_B.grad, model_torch.w_lora_B.grad))

    x_test = torch.rand(3, 5)
    model_lib.eval()
    model_torch.eval()
    print(torch.isclose(model_lib(x_test), model_torch(x_test)))

tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True]])
tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True